In [1]:
import pandas as pd

import numpy as np

import os

import matplotlib.pyplot as plt

from itertools import chain

In [2]:
class MurderWallAsset:
    def __init__(self, data_frame, file_path, metadata):
        self.data_frame = data_frame
        self.file_path = file_path
        self.metadata = metadata

    def __repr__(self):
        return self.metadata.__repr__()

class TrialData:
    def __init__(self, subject_id, condition_id, activity_id):
        self.subject = subject_id
        self.condition = condition_id
        self.activity = activity_id

    def __repr__(self):
        return f"Subject {self.subject} | Condition {self.condition} | Activity {self.activity}"

class TrialDataPath:
    def __init__(self, path, d_types, trial_data):
        self.path = path
        self.d_types = d_types
        self.trial_data = trial_data

    def _create_path(self, d_type):
        return f"{self.path}/{self.trial_data.subject}/{self.trial_data.condition}/processed-{d_type}-{self.trial_data.activity}.csv"
    
    def _assert_existence_of_new_data(self, emg, force):
        assert(os.path.exists(emg))
        assert(os.path.exists(force))

    def _get(self):
        emg, force = self.d_types
        emg_path = self._create_path(emg)
        force_path = self._create_path(force)
        self._assert_existence_of_new_data(emg_path, force_path)
        return (emg_path, force_path)
    
    def get_path(self):
        emg, force = self._get()
        return (emg, force)

    def get_data_frame(self):
        emg, force = self._get()
        print(f"Converting {emg} to dataframe")
        print(f"Converting {force} to dataframe")
        return (pd.read_csv(emg, index_col=0), pd.read_csv(force, index_col=0))

class MurderWall:
    def __init__(self, subject_ids, get_data_frames = False):
        self.get_data_frames = get_data_frames
        self.subjects = subject_ids
        self.activities = [f"0{n}" for n in range(1, 9)]
        self.conditions = ["01", "02"]
        self.data_types = ["emg", "force"]

    def _create_condition_column(self, activity_data):
        data_path = TrialDataPath("./proc_data", self.data_types, activity_data)
        if self.get_data_frames:
            return MurderWallAsset(data_path.get_data_frame(), data_path.get_path(), activity_data)
        else:
            return TrialDataPath("./proc_data", self.data_types, activity_data).get_path()
    
    def _create_activity_column(self, subject_id, activity_id):
        activity_data = [TrialData(subject_id, condition_id, activity_id) for condition_id in self.conditions]
        return [self._create_condition_column(data) for data in activity_data]
    
    def _create_subject_row(self, subject_id):
        return list(chain.from_iterable([self._create_activity_column(subject_id, activity_id) for activity_id in self.activities]))
        
    def create_layout(self):
        return MurderWallDetective(np.array([self._create_subject_row(subject_id) for subject_id in self.subjects], dtype=object))

class MurderWallDetective:
    def __init__(self, murder_wall):
        self.murder_wall = murder_wall
        self.rows = len(murder_wall)
        self.columns = len(murder_wall[0])

    def get_activity(self, index):
        assert(self.columns > index)
        return self.murder_wall[:, index]

    def get_subject(self, index):
        assert(self.rows > index)
        return self.murder_wall[index, :]

    def get_columns(self):
        return self.murder_wall[0][0].data_frame[0].columns

In [3]:
def get_subjects(subject_data_path):
    subjects = os.listdir(subject_data_path)
    assert(len(subjects) == 12)
    return subjects

In [4]:
def clip_values_(df_to_be_clipped, df_to_clip_to):
        return df_to_be_clipped[df_to_be_clipped.index <= df_to_clip_to.last_valid_index()]

def sort_on(frame):
    _, shape = frame
    rows, _ = shape
    return rows

def clip_values(dfs: [pd.DataFrame]):
    index, _ = min(enumerate([frame.shape for frame in dfs]), key = sort_on)
    return [clip_values_(frame, dfs[index]) for frame in dfs]

In [5]:
murder_wall = MurderWall(get_subjects("./proc_data"), get_data_frames=True)
data = murder_wall.create_layout()

Converting ./proc_data/07/01/processed-emg-01.csv to dataframe
Converting ./proc_data/07/01/processed-force-01.csv to dataframe
Converting ./proc_data/07/02/processed-emg-01.csv to dataframe
Converting ./proc_data/07/02/processed-force-01.csv to dataframe
Converting ./proc_data/07/01/processed-emg-02.csv to dataframe
Converting ./proc_data/07/01/processed-force-02.csv to dataframe
Converting ./proc_data/07/02/processed-emg-02.csv to dataframe
Converting ./proc_data/07/02/processed-force-02.csv to dataframe
Converting ./proc_data/07/01/processed-emg-03.csv to dataframe
Converting ./proc_data/07/01/processed-force-03.csv to dataframe
Converting ./proc_data/07/02/processed-emg-03.csv to dataframe
Converting ./proc_data/07/02/processed-force-03.csv to dataframe
Converting ./proc_data/07/01/processed-emg-04.csv to dataframe
Converting ./proc_data/07/01/processed-force-04.csv to dataframe
Converting ./proc_data/07/02/processed-emg-04.csv to dataframe
Converting ./proc_data/07/02/processed-fo

In [7]:
def plot_and_save(condition_data)
    plt.figure(figsize=(20,10))
    condition_data.boxplot(showfliers=False)
    plt.title(condition_1.metadata)
    plt.savefig(f"./test_data/{condition_1.metadata}.png")
    plt.close()

for i in range(0, data.columns, 2):
    condition_1_data = pd.DataFrame(columns=data.get_columns())
    condition_2_data = pd.DataFrame(columns=data.get_columns())
    for condition_1, condition_2 in zip(data.get_activity(i), data.get_activity(i+1)):
        condition_1_area = condition_1.data_frame[0].cumsum()
        condition_2_area = condition_2.data_frame[0].cumsum()
        condition_1_data.loc[len(condition_1_data)] = condition_1_area.iloc[-1]
        condition_2_data.loc[len(condition_2_data)] = condition_2_area.iloc[-1]
    
    plt.figure(figsize=(20, 10))
    condition_2_data.boxplot(showfliers=False)
    plt.title(condition_2.metadata)
    plt.savefig(f"./test_data/{condition_2.metadata}.png")
    plt.close()

Subject 03 | Condition 02 | Activity 01
Subject 03 | Condition 02 | Activity 02
Subject 03 | Condition 02 | Activity 03
Subject 03 | Condition 02 | Activity 04
Subject 03 | Condition 02 | Activity 05
Subject 03 | Condition 02 | Activity 06
Subject 03 | Condition 02 | Activity 07
Subject 03 | Condition 02 | Activity 08


In [4]:
data = pd.read_csv("../proc_data/02/01/processed-emg-01.csv", index_col=0)

In [17]:
data.head().iloc[:, 3:]

,BB,PD,AD,MD,TR,FL,EX,BR
6090,4.917742,1.366727,0.054041,0.248547,0.292419,0.151889,0.047073,1.904489
6091,4.917742,1.362161,0.054030,0.248100,0.292251,0.152129,0.047045,1.906190
6092,4.917742,1.353109,0.054041,0.247728,0.292133,0.152403,0.047027,1.906452
6093,4.917742,1.348839,0.054039,0.247198,0.292164,0.152415,0.047026,1.908292
6094,4.917742,1.348791,0.053987,0.247080,0.292294,0.152435,0.047013,1.908560


In [34]:
print(len(data.iloc[:, 3]))
n_factor = 10000
(data.iloc[:n_factor, 3:].cumsum().iloc[-1]) / n_factor

56090


BB    4.877005
PD    1.404598
AD    0.051281
MD    0.222364
TR    0.281575
FL    0.154541
EX    0.159421
BR    1.911963
Name: 16089, dtype: float64